In [1]:
from dataclasses import dataclass
from typing import Union, Any

Level = Union[int, str]


class RichTypeError(TypeError):
    pass


@dataclass(frozen=True, slots=True)
class Arg:
    name: str


@dataclass(frozen=True, slots=True)
class Val:
    value: Any


def do_shit(level):
    raise RichTypeError(
        "Expected",
        Arg("level"),
        "to be",
        Level,
        "given",
        type(level),
        Val(level),
    )


In [2]:
from numbers import Number
from typing import Annotated, Optional, get_args, get_origin, get_type_hints
from splatlog.lib.typeguard import satisfies
from typeguard import check_type
import dataclasses


@dataclasses.dataclass(frozen=True)
class ValueRange:
    min: Optional[Number] = None
    max: Optional[Number] = None

    def is_valid(self, value: Number) -> bool:
        if self.min is not None and value < self.min:
            return False
        if self.max is not None and value > self.max:
            return False
        return True


Verbosity = Annotated[int, ValueRange(min=0)]

args = get_args(Verbosity)


def my_check_type(name, value, expected_type):
    check_type(name, value, expected_type)

    if get_origin(expected_type) is Annotated:
        for arg in get_args(expected_type):
            if isinstance(arg, ValueRange) and not arg.is_valid(value):
                raise TypeError(
                    "type of {} must be {}; got {} instead".format(
                        name, expected_type, type(value)
                    )
                )


my_check_type("verbosity", -1, Verbosity)


TypeError: type of verbosity must be typing.Annotated[int, ValueRange(min=0, max=None)]; got <class 'int'> instead

In [ ]:
from collections import defaultdict


def tree_default_factory():
    return defaultdict(tree_default_factory)


def tree_build(entries):
    tree = defaultdict(tree_default_factory)
    for (*key_path, final_key), value in entries:
        target = tree
        for key in key_path:
            target = target[key]
        target[final_key] = value
    return tree


tree_build(
    (
        (("a", "b"), 1),
        (("a", "c"), 2),
        (("a", "d", "e"), 3),
    )
)


defaultdict(<function __main__.tree_default_factory()>,
            {'a': defaultdict(<function __main__.tree_default_factory()>,
                         {'b': 1,
                          'c': 2,
                          'd': defaultdict(<function __main__.tree_default_factory()>,
                                      {'e': 3})})})

In [11]:
import timeit


def f(x):
    return x


def g(*x):
    return x


def h1(*x):
    return g(x)


def h2(*x):
    return g(*x)


def tit(fn, *args):
    print(
        "{}({}) {}".format(
            fn.__name__,
            ", ".join(repr(a) for a in args),
            timeit.timeit(lambda: fn(*args)),
        )
    )


tit(f, 123)
tit(g, 123, 456)
tit(h1, 123, 456, 789)
tit(h2, 123, 456, 789)


f(123) 0.14334273600252345
g(123, 456) 0.14037512097274885
h1(123, 456, 789) 0.21693217003485188
h2(123, 456, 789) 0.23705493501620367


In [55]:
from splatlog.lib.collections.peek_iterator import PeekIteratorWrapper

def gen(n):
    for _ in range(n):
        yield f"gen {n}"

def gen_wrap(n):
    return PeekIteratorWrapper(gen(n))

def lst(n):
    result = []
    for _ in range(n):
        result.append(f"lst {n}")
    return result

def run(fn, n):
    rs = fn(n)
    if isinstance(rs, PeekIteratorWrapper):
        return rs.is_empty()
    else:
        return len(rs) == 0


tit(run, gen_wrap, 9)
tit(run, lst, 9)


run(<function gen_wrap at 0x10bfc2c20>, 9) 2.358833139995113
run(<function lst at 0x10bfc3130>, 9) 2.312404152995441


In [25]:
from functools import wraps
from io import StringIO
from itertools import chain
import string
from textwrap import wrap
from sys import stdout


class Element:
    def __init__(self, fn, args, kwds):
        self._fn = fn
        self._args = args
        self._kwds = kwds

    def __repr__(self):
        return "{}({})".format(
            self._fn.__name__,
            ", ".join(
                chain(
                    (repr(x) for x in self._args),
                    (f"{k!r}={v!r}" for k, v in self._kwds),
                )
            ),
        )

    def __str__(self):
        return self.render()

    def print(self, file=stdout):
        self._fn(*self._args, file=file, **self._kwds)

    def render(self):
        sio = StringIO()
        self.print(file=sio)
        return sio.getvalue()


def element(fn):
    @wraps(fn)
    def constructor(*args, **kwds):
        return Element(fn, args, kwds)

    return constructor


@element
def li(text, file=stdout, indent_level=0, marker="-", tab_width=4):
    indent = " " * tab_width * indent_level
    for line in wrap(
        text,
        width=80,
        initial_indent=(
            indent
            + "{marker:{tab_width}}".format(marker=marker, tab_width=tab_width)
        ),
        subsequent_indent=(indent * 2),
        tabsize=4,
    ):
        print(line, file=file)


@element
def ol(*items, file=stdout, marker="-", tab_width=4):
    for item in items:
        li(item, marker=marker, tab_width=tab_width).print(file=file)


class Formatter(string.Formatter):
    def convert_field(self, value, conversion):
        if conversion == "b":
            sio = StringIO()
            sio.write("\n")
            value.print(file=sio)
            return sio.getvalue()
        return super().convert_field(value, conversion)


formatter = Formatter()


def p(template, *args, **kwds):
    print(formatter.format(template, *args, **kwds))


p("any of: {failures!b}", failures=ol("blah", "blow me", "I like cats"))


any of: 
-   blah
-   blow me
-   I like cats



In [56]:
from abc import ABCMeta, abstractmethod
from functools import wraps
from io import StringIO
from itertools import chain
import string
from textwrap import wrap
from sys import stdout


class Element(metaclass=ABCMeta):
    @classmethod
    def wrap(cls, child, **opts):
        if isinstance(child, cls):
            return child
        return cls(child, **opts)

    def __repr__(self):
        return "{}({})".format(
            self._fn.__name__,
            ", ".join(
                chain(
                    (repr(x) for x in self._args),
                    (f"{k!r}={v!r}" for k, v in self._kwds),
                )
            ),
        )

    def __str__(self):
        return self.render()

    @abstractmethod
    def print_block(self, file=stdout, indent_level=0):
        raise NotImplementedError("abstract")

    @abstractmethod
    def print_inline(self, file=stdout):
        raise NotImplementedError("abstract")

    def print(self, file=stdout, display="inline", **kwds):
        if display == "inline":
            self.print_inline(file, **kwds)
        elif display == "block":
            self.print_block(file, **kwds)
        else:
            raise ValueError(
                "Expected 'inline' or 'block', given {}: {!r}".format(
                    type(display),
                    display,
                )
            )

    def render(self, display="inline"):
        sio = StringIO()
        self.print(file=sio, display=display)
        return sio.getvalue()


class li(Element):
    def __init__(self, *children, marker="-", tab_width=4):
        self._children = children
        self._marker = marker
        self._tab_width = tab_width

    def print_block(self, file=stdout, indent_level=0):
        indent = " " * self._tab_width * indent_level
        for child in self._children:
            if isinstance(child, Element):
                child.print_block()
        for line in wrap(
            self._children,
            width=80,
            initial_indent=(
                indent
                + "{marker:{tab_width}}".format(
                    marker=self._marker, tab_width=self._tab_width
                )
            ),
            subsequent_indent=(indent * 2),
            tabsize=4,
        ):
            print(line, file=file)

    def print_inline(self, file=stdout):
        file.write(self._children)


class ul(Element):
    def __init__(self, *children, marker="-", tab_width=4):
        self._children = [
            li.wrap(child, marker="-", tab_width=4) for child in children
        ]
        self._marker = marker
        self._tab_width = tab_width

    def print_block(self, file=stdout, indent_level=0):
        for child in self._children:
            child.print_block(file=file, indent_level=indent_level)

    def print_inline(self, file=stdout):
        for index, child in enumerate(self._children):
            if index != 0:
                file.write(", ")
            child.print_inline(file=file)


class Formatter(string.Formatter):
    def __init__(self, display):
        super().__init__()
        self._display = display

    def convert_field(self, value, conversion):
        if conversion == "l":
            el = ul(*value)
            sio = StringIO()
            if self._display == "block":
                sio.write("\n")
                el.print_block(file=sio)
            else:
                el.print_inline(file=sio)
            return sio.getvalue()

        if conversion == "b":
            sio = StringIO()
            sio.write("\n")
            value.print_block(file=sio)
            return sio.getvalue()

        if conversion == "i":
            sio = StringIO()
            value.print_inline(file=sio)
            return sio.getvalue()

        return super().convert_field(value, conversion)


b_fmt = Formatter(display="block")
i_fmt = Formatter(display="inline")


def p(formatter, template, *args, **kwds):
    print(formatter.format(template, *args, **kwds))


items = ("blah", "blow me", "I like cats")
ul_1 = ul(*items)

p(b_fmt, "any of: {items!l}", items=items)
p(i_fmt, "any of: {items!l}", items=items)


AttributeError: 'tuple' object has no attribute 'expandtabs'

In [63]:
import textwrap
import re

def split(text, width):
    # if m := re.search(r'\W', text):
    #     if m.start
    bite = textwrap.shorten(text, width, placeholder='')
    rest = text[len(bite):]
    return (bite, rest)

split("  Hello world!", 5)

('Hello', 'lo world!')

In [90]:
def split_2(text, width):
    matches = textwrap.TextWrapper.wordsep_re.findall(text)
    index = 0
    length = 0
    while index < len(matches) and length <= width:
        length += len(matches[index])
        index += 1
    if length > width:
        index -= 1
    return (matches[:index], matches[index:])

text = ""

for width in range(len(text) + 2):
    print("width", width, ":", [(s, len(s)) for s in ("".join(s) for s in split_2(text, width))])


width 0 : [('', 0), ('', 0)]
width 1 : [('', 0), ('', 0)]


In [72]:
textwrap.TextWrapper.wordsep_re.findall("a")

['a']